In [ ]:
%pip install -r requirements.txt

In [41]:
from game import GameOf2048
env = GameOf2048()

In [42]:
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, truncated, info = env.step(action)
        score+=reward
    print('Episode:{}, Score:{}'.format(episode, score))

Episode:1, Score:1116.0
Episode:2, Score:1620.0
Episode:3, Score:368.0
Episode:4, Score:1356.0
Episode:5, Score:1068.0
Episode:6, Score:672.0
Episode:7, Score:708.0
Episode:8, Score:560.0
Episode:9, Score:1360.0
Episode:10, Score:1340.0
